<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Train Churn Model for Banking Customer data using Clustering and in-DB Functions
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>

<center><img src="images/customer_group.png" width=600px; style="border: 4px solid #404040; border-radius: 10px;"/></center>
<br>
<p style = 'font-size:16px;font-family:Arial'>Customer churn is a critical metric in banking because it can directly impact a bank's revenue and profitability. When customers leave, banks lose the income they would have earned from those customers' transactions, investments, and account fees. Additionally, attracting new customers to replace those who have left can be expensive and time-consuming, so reducing customer churn is often more cost-effective than acquiring new customers.</p>

<p style = 'font-size:16px;font-family:Arial'>Customer churn can also be an indicator of customer satisfaction and loyalty. If customers leave at a high rate, they may be dissatisfied with the bank's products or services, customer service, or overall experience.</p>

<p style = 'font-size:16px;font-family:Arial'>Banks can use various strategies to reduce customer churns, such as improving customer service, offering more competitive rates and fees, providing personalized recommendations and offers, and enhancing digital channels and mobile apps. By tracking and analyzing customer churn rates, banks can identify areas for improvement and make strategic decisions to retain customers and improve overall customer satisfaction.</p>

<p style = 'font-size:16px;font-family:Arial'>In this demo, we demonstrate how to implement the entire lifecycle of churn prediction can using Vantage technologies and, specifically, the combination of Bring Your Own Model (BYOM), Vantage Analytics Library (VAL) and teradataml python client library solution.</p>

<p style = 'font-size:18px;font-family:Arial'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>Initiate a connection to Vantage</li>
    <li>Run K-Means on the Embeddings Store and then build final table with Cluster ID assignments to rows</li>
    <li>Data Transformation</li>
    <li>Modelling</li>
    <li>Evaluate the Model</li>
    <li>Cleanup</li>
</ol>

<hr style="height:2px;border:none">
<p style = 'font-size:18px;font-family:Arial'><b>Import the required libraries</b></p>

<p style = 'font-size:16px;font-family:Arial'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

<div class="alert alert-block alert-warning">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>Please ensure that <b>Step1_Banking_Customer_Churn_Sentiment_Analysis</b> is executed before running this file.</i></p>
</div>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd

import teradataml as tdml
import getpass
from teradataml import in_schema
from teradataml import DecisionForest, XGBoost, TrainTestSplit, DecisionForestPredict, XGBoostPredict, SentimentExtractor, ColumnTransformer, ScaleFit, OneHotEncodingFit
from teradataml import ColumnSummary, AutoML, AutoClassifier
from teradataml import RoundColumns, ClassificationEvaluator, ROC
from teradataml import (
    DataFrame,
    execute_sql
)
from teradataml import KMeans
from teradataml import create_context
from teradataml import SVM, SVMPredict
from teradataml import GridSearch, RandomSearch
from teradatasqlalchemy import BYTEINT
display.max_rows = 5

In [ ]:
tdml.configure.val_install_location = "val"

<hr style="height:2px;border:none">
<b style = 'font-size:20px;font-family:Arial'>1. Initiate a connection to Vantage</b>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
# Change host and/or username as needed
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

execute_sql('''SET query_band='DEMO=Step3_Train_Churn_Model_with_Clustering.ipynb;' UPDATE FOR SESSION; ''')

<p style = 'font-size:20px;font-family:Arial'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_BankChurnIVSM_cloud');"  
%run -i ../run_procedure.py "call get_data('DEMO_BankChurnIVSM_local');"

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"        # Takes 10 seconds

In [ ]:
df = tdml.DataFrame('complaint_embeddings_store')

In [ ]:
df.head()

<hr style="height:2px;border:none">
<b style = 'font-size:20px;font-family:Arial'>2. Run K-Means on the Embeddings Store and then build final table with Cluster ID assignments to rows</b>

<p style = 'font-size:16px;font-family:Arial'>The <b>K-means()</b> function groups a set of observations into k clusters in which each observation belongs to the cluster with the nearest mean (cluster centers or cluster centroid). This algorithm minimizes the objective function, that is, the total Euclidean distance of all data points from the center of the cluster</p>

In [ ]:
cols = list(df.columns)[2:]

KMeans_out = KMeans(id_column="CustomerId",
                    target_columns=cols,
                    data=df,
                    num_clusters=10,
                    output_cluster_assignment=True
                    )

<p style = 'font-size:16px;font-family:Arial'>The output below shows cluster assignment for each row.</p>

In [ ]:
clusters = KMeans_out.result

<p style = 'font-size:16px;font-family:Arial'>Let's check how many data points each cluster has.</p>

In [ ]:
clusters

In [ ]:
merged_df = clusters.merge(df[['CustomerId','Customer_Complaint']], on='CustomerId', how='inner', lsuffix='_left', rsuffix='_right')

In [ ]:
merged_df=merged_df.drop('CustomerId__left', axis=1)

<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage.</p>
<p style = 'font-size:16px;font-family:Arial'><b><i>*Please scroll down to the end of the notebook for detailed column descriptions of the dataset.</i></b></p>

In [ ]:
customer_churn = DataFrame(in_schema('DEMO_BankChurnIVSM', 'Bank_Churn'))
customer_churn

In [ ]:
new_df = customer_churn.merge(merged_df[['CustomerId__right','td_clusterid_kmeans']],
                              on='customerid = CustomerId__right',
                              how='inner')
new_df

In [ ]:
new_df = new_df.drop('CustomerId__right',axis=1)

In [ ]:
new_df

<hr style="height:2px;border:none">
<b style = 'font-size:20px;font-family:Arial'>3. Data Transformation</b>

In [ ]:
target_variable = "Exited"
numeric_columns = ["Age", "Balance", "CreditScore", "EstimatedSalary", "Tenure"]
categorical_columns = ["Gender", "Geography", "td_clusterid_kmeans", "NumOfProducts"]
binary_columns = ["HasCrCard", "IsActiveMember"]
id_column = ["CustomerId"]

<p style = 'font-size:16px;font-family:Arial'><b>ScaleFit()</b> function outputs statistics to input to ScaleTransform() function, which scales specified input DataFrame columns.<br>

In [ ]:
fit1 = ScaleFit(data=new_df,
                target_columns=numeric_columns,
                scale_method="USTD",
                miss_value="KEEP",
                global_scale=False,
                multiplier="1")

<p style = 'font-size:16px;font-family:Arial'><b>OneHotEncodingFit </b>outputs a table of attributes and categorical values to input to OneHotEncodingTransform which encodes them as one-hot numeric vectors.</p>

In [ ]:
fit2 = OneHotEncodingFit(data=new_df,
                         is_input_dense=True,
                         approach="auto",
                         target_column=categorical_columns[0:2],
                         category_counts=[2,3])

<p style = 'font-size:16px;font-family:Arial'>The <b>ColumnTransformer</b> function transforms the entire dataset in a single operation. You only need
to provide the FIT tables to the function, and the function runs all transformations that you require in a
single operation. Running all the it table transformations together in one-go gives approx. 30% performance improvement over running each transformation sequentially.</p>

In [ ]:
new_table = ColumnTransformer(input_data=new_df,
                             onehotencoding_fit_data=fit2.result,
                             scale_fit_data=fit1.output).result

In [ ]:
new_table=new_table[['CustomerId', 'Age', 'Balance', 'CreditScore', 'EstimatedSalary', 'Exited', 'Gender', 'Geography', 'HasCrCard',
                     'IsActiveMember', 'NumOfProducts', 'Tenure', 'td_clusterid_kmeans', 'Gender_0', 'Gender_1', 'Geography_0',
                     'Geography_1', 'Geography_2']]

<p style = 'font-size:18px;font-family:Arial'><b>3.1 Train-Test Split</b>

<p style = 'font-size:16px;font-family:Arial'>The <b>TrainTestSplit()</b> function divides the dataset into train and test subsets to be used for evaluating machine learning models and validation processes.<br>
80% is used for Training and 20% for validation.</p>

In [ ]:
TrainTestSplit_out = TrainTestSplit(data = new_table,
                                    id_column='CustomerId',
                                    train_size=0.80,
                                    test_size=0.20,
                                    seed=3432)

In [ ]:
TrainTestSplit_out.result.head()

In [ ]:
df_train = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 1].drop(['TD_IsTrainRow'], axis = 1)
df_test = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 0].drop(['TD_IsTrainRow'], axis = 1)

print("Training Set = " + str(df_train.shape[0]) + ". Testing Set = " + str(df_test.shape[0]))

In [ ]:
tdml.copy_to_sql(df_train, table_name = 'clean_data_train1', if_exists = 'replace')
tdml.copy_to_sql(df_test, table_name = 'clean_data_test1', if_exists = 'replace')

In [ ]:
df_train = tdml.DataFrame(in_schema('demo_user','clean_data_train1'))

In [ ]:
df_test = tdml.DataFrame(in_schema('demo_user','clean_data_test1'))

<hr style="height:2px;border:none">
<p style = 'font-size:20px;font-family:Arial'><b>4. Modelling</b></p>

<p style = 'font-size:18px;font-family:Arial'><b>4.1 Train an XGBoost Model</b>
<p style = 'font-size:16px;font-family:Arial'>The <b>XGBoost()</b> function is an efficient implementation of gradient boosting for classification and regression tasks. It builds an ensemble of decision trees in a sequential manner to minimize prediction error.</p>


In [ ]:
formula_str = "Exited ~ CreditScore + Age + Tenure + Balance + NumOfProducts + HasCrCard + IsActiveMember + EstimatedSalary + Gender_0 + Gender_1 + Geography_0 + Geography_1 + Geography_2 + td_clusterid_kmeans"

In [ ]:
XGBoost_out2 = XGBoost(data=df_train,
                       id_column='CustomerId',
                       loss_function='logistic',
                       formula = formula_str,
                       iter_num=5,
                       min_node_size=1,
                       #num_boosted_trees=50,                       
                       num_boosted_trees=80,
                       lambda1 = 500,
                       shrinkage_factor=0.5,
                       max_depth=10)

<p style = 'font-size:18px;font-family:Arial'><b>4.2 Predict Labels using the XGBoost Model</b></p>

<p style = 'font-size:16px;font-family:Arial'>The <b>XGBoostPredict()</b> function is used to predict the target labels for the test dataset (<b>df_test</b>) based on the trained XGBoost model. </p>

In [ ]:
XGBoostPredict_out_1 = XGBoostPredict(newdata=df_test,
                                      object=XGBoost_out2.result,
                                      id_column='CustomerId',
                                     accumulate='Exited')

In [ ]:
XGBoostPredict_out_1.result

In [ ]:
predict_df = XGBoostPredict_out_1.result
predict_df = predict_df.assign(Prediction = predict_df.Prediction.cast(type_ = BYTEINT))
predict_df.head()

<p style = 'font-size:20px;font-family:Arial'><b>5. Evaluate the Model</b></p>
<p style = 'font-size:16px;font-family:Arial'> <b>ClassificationEvaluator()</b> function evaluates and emits various metrics of classification model based on its predictions on the data. Apart from accuracy, the secondary output data returns micro, macro, and weighted-averaged metrics of precision, recall, and F1-score values.<br>
This is a powerful function, and doesn't move data outside Vantage.

In [ ]:
ClassificationEvaluator_obj = ClassificationEvaluator(data=predict_df,
                                                          observation_column='Exited',
                                                          prediction_column='Prediction',
                                                          labels=['0', '1'])
classeval_decisiondf = ClassificationEvaluator_obj.output_data
classeval_decisiondf

<p style = 'font-size:18px;font-family:Arial'><b>5.1 Compute ROC Curve</b></p>
<p style = 'font-size:16px;font-family:Arial'>The <b>ROC()</b> function calculates the Receiver Operating Characteristic (ROC) curve to evaluate the performance of the model, using the predicted probabilities and the actual class labels.

In [ ]:
roc_df = ROC(data = predict_df, 
                    probability_column = "Prediction",
                    observation_column = "Exited",
                    positive_class="1"
                    )
roc_df.output_data

In [ ]:
auc = roc_df.result.get_values()[0][0]
auc

<p style = 'font-size:18px;font-family:Arial'><b>5.2 Plot ROC Curve</b></p>
<p style = 'font-size:16px;font-family:Arial'>Plots the ROC curve using fpr (False Positive Rate) and tpr (True Positive Rate) from the ROC data, and displays the Area Under the Curve (AUC) for model evaluation.

In [ ]:
plot_roc_df = roc_df.output_data
plot =  plot_roc_df.plot(x=plot_roc_df.fpr, y=plot_roc_df.tpr,
                         title="Receiver Operating Characteristic (ROC) Curve",
                         xlabel='False Positive Rate', 
                         ylabel='True Positive Rate', 
                         color="blue",
                         legend=f'AUC = {round(auc, 4)}',
                         legend_style='lower right',
                         grid_linestyle='--',
                         grid_linewidth=0.5)
 
# Display the plot.
plot.show()

<p style = 'font-size:18px;font-family:Arial'><b>5.3 Hyperparameter Tuning</b></p>
<p style = 'font-size:16px;font-family:Arial'>Sets the parameters for the classification model, including input columns, response column, hyperparameters (e.g., max_depth, lambda1), and other settings such as shrinkage_factor, seed, and iter_num.

In [ ]:
model_params = {"input_columns":['CreditScore','Age', 'Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary','Gender_0','Gender_1','Geography_0','Geography_1','Geography_2','td_clusterid_kmeans'],
                    "response_column" :'Exited',
                    "max_depth":(5,10,15),
                    "lambda1" :(1000.0,0.001),
                    "model_type" :"Classification",
                    "seed":32,
                    "shrinkage_factor":0.1,
                    "iter_num":(5, 50)}

In [ ]:
eval_params = {"id_column": "CustomerId",
               "accumulate":"Exited",
               "model_type":'Classification',
               "object_order_column":['task_index', 'tree_num', 'iter','class_num', 'tree_order']}

<p style = 'font-size:16px;font-family:Arial'><b>GridSearch</b> is an exhaustive search algorithm that covers all possible parameter values to identify optimal hyperparameters. It works for teradataml analytic functions from SQLE, BYOM, VAL and UAF features.teradataml GridSearch allows user to perform hyperparameter tuning for all model trainer and non-model trainer functions.

In [ ]:
gs_obj = GridSearch(func=XGBoost, params=model_params)

In [ ]:
gs_obj.fit(data=df_train, verbose=2, run_parallel=True, evaluation_metric='Accuracy', **eval_params)

In [ ]:
gs_obj.models

In [ ]:
gs_obj.model_stats

<p style = 'font-size:16px;font-family:Arial'>Function uses model training function generated models from SQLE, 
    VAL and UAF features for predictions. Predictions are made using 
    the best trained model. Predict function is not supported for
    non-model trainer function.

In [ ]:
gs_pred = gs_obj.predict(newdata=df_test, **eval_params)

In [ ]:
print("Prediction Result: \n", gs_pred.result)

In [ ]:
gs_obj.best_params_

In [ ]:
roc_df = ROC(data = gs_pred.result, 
                    probability_column = "Prediction",
                    observation_column = "Exited",
                    positive_class="1"
                    )
auc = roc_df.result.get_values()[0][0]
print('AUC: ', auc)

plot_roc_df = roc_df.output_data
plot =  plot_roc_df.plot(x=plot_roc_df.fpr, y=plot_roc_df.tpr,
                         title="Receiver Operating Characteristic (ROC) Curve",
                         xlabel='False Positive Rate', 
                         ylabel='True Positive Rate', 
                         color="blue",
                         legend=f'AUC = {round(auc, 4)}',
                         legend_style='lower right',
                         grid_linestyle='--',
                         grid_linewidth=0.5)
 
# Display the plot.
plot.show()

<hr style="height:2px;border:none">
<b style = 'font-size:20px;font-family:Arial'>6. Cleanup</b>
<p style = 'font-size:16px;font-family:Arial'>The following code will remove the context.</p>

In [ ]:
tdml.remove_context()

<hr style="height:1px;border:none;">
<b style = 'font-size:18px;font-family:Arial'>Dataset:</b>

- `Unnamed`: Unnamed
- `CustomerId`: Customer ID
- `Surname`: Surname
- `CreditScore`: Credit score
- `Geography`: Country (Germany / France / Spain)
- `Gender`: Gender (Female / Male)
- `Age`: Age
- `Tenure`: No of years the customer has been associated with the bank
- `Balance`: Balance
- `NumOfProducts`: No of bank products used
- `HasCrCard`: Credit card status (0 = No, 1 = Yes)
- `IsActiveMember`: Active membership status (0 = No, 1 = Yes)
- `EstimatedSalary`: Estimated salary
- `Exited`: Abandoned or not? (0 = No, 1 = Yes)

<p style = 'font-size:16px;font-family:Arial'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
    <li>KMeans reference: <a href = 'https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/MODEL-TRAINING-functions/KMeans'>here</a></li>
    <li>KMeansPredict reference: <a href = 'https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/MODEL-SCORING-functions/KMeansPredict'>here</a></li>
    <li>SVM reference: <a href = 'https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/MODEL-TRAINING-functions/SVM'>here</a></li>
    <li>SVMPredict reference: <a href = 'https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/MODEL-SCORING-functions/SVMPredict'>here</a></li>
    <li>XGBoost reference: <a href = 'https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/MODEL-TRAINING-functions/XGBoost'>here</a></li>
    <li>XGBoostPredict reference: <a href = 'https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/MODEL-SCORING-functions/XGBoostPredict'>here</a></li>
    <li>ScaleFit reference: <a href = 'https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/FEATURE-ENGINEERING-TRANSFORM-functions/ScaleFit'>here</a></li>
    <li>OneHotEncodingFit reference: <a href = 'https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/FEATURE-ENGINEERING-TRANSFORM-functions/OneHotEncodingFit'>here</a></li>
    <li>TrainTestSplit reference: <a href = 'https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/MODEL-EVALUATION-functions/TrainTestSplit'>here</a></li>
    <li>ClassificationEvaluator reference: <a href = 'https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/MODEL-EVALUATION-functions/ClassificationEvaluator'>here</a></li>
    <li>ColumnTransformer reference: <a href = 'https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/FEATURE-ENGINEERING-TRANSFORM-functions/ColumnTransformer'>here</a></li>
    <li>ROC reference: <a href = 'https://docs.teradata.com/r/Enterprise/Teradata-Package-for-Python-Function-Reference-17.20/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/MODEL-EVALUATION-functions/ROC'>here</a></li>
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2025,2026. All Rights Reserved
        </div>
    </div>
</footer>